# Convolutional Neural Networks (CNNs) Assignment

#### 1) Load a pretrained network from TensorFlow Hub
[ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) is a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt).

In [ ]:
from PIL import Image, ImageOps
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
module = hub.Module("https://tfhub.dev/google/imagenet/"
                    "resnet_v1_50/classification/1")

In [ ]:
height, width = hub.get_expected_image_size(module)
print(height, width)

#### 2) Apply it to classify the images downloaded below (images from a search for animals in national parks).

In [ ]:
# !pip install google_images_download

In [ ]:
from typing import List
from google_images_download import google_images_download


def resize_image(filename, new_width=224, new_height=224):
    pil_image = Image.open(filename)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert('RGB')
    pil_image_rgb.save(filename, format='JPEG', quality=90)


def image_input_fn(image_paths: List[str]):
    filename_queue = tf.train.string_input_producer(
        image_paths, shuffle=False)
    _, value = tf.WholeFileReader().read(filename_queue)
    image_tf = tf.image.decode_jpeg(value, channels=3)

    return tf.image.convert_image_dtype(image_tf, tf.float32)


def download_images(keywords="animal national park", limit=20, print_urls=False):
    response = google_images_download.googleimagesdownload()
    arguments = dict(keywords=keywords,
                     limit=limit,
                     print_urls=print_urls)
    response.download(arguments)


def process_img_path(img_path):
    return image.load_img(img_path)

In [ ]:
download_images(keywords="animal national park", limit=20, print_urls=False)

In [ ]:
str(path)

In [ ]:
from pathlib import Path

image_paths = []
p = Path('downloads/animal national park')
for path in p.iterdir():
    if image_path.endswith(('.jpg', '.jpeg')):
        try:
            resize_image(str(path))
            image_paths.append(str(path))
        except OSError:
            pass

In [ ]:
image_paths

In [ ]:
for img in image_paths:
    x = image.img_to_array(process_img_path(image_paths[0]))
    print(x.shape)

In [ ]:
y = tf.data.Dataset.from_tensor_slices(x)

In [ ]:
x = image_input_fn(image_paths) # -> to batch?

In [ ]:
x.shape

In [ ]:
x.set_shape([len(image_paths), 224, 224, 3])

In [ ]:
x.shape

In [ ]:
module(y)

### Could not get `tensorflow_hub` module to work, let's try to the built-in model in `keras`

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

# Pre-trained model

PTModel = ResNet50(weights='imagenet')

# Take an image instance, turn into an np array with float32,
# match the shape of the image to the shape of the pretrained model
# preprocess the data
# use pre trained model to classify image
# use decode_predictions to eval perfomance

for path in paths:
    ImgPath = path
    Img = image.load_img(ImgPath, target_size=(224, 224))
    InputIMG = image.img_to_array(Img)
    InputIMG = np.expand_dims(InputIMG, axis=0)
    InputIMG = preprocess_input(InputIMG)
    PredData = PTModel.predict(InputIMG)
    print('Predicted:', decode_predictions(PredData, top=2)[0])

#### Report both the most likely estimated class for any image, and also investigate (a) images where the classifier isn't that certain (the best estimate is low), and (b) images where the classifier fails.

#### Answer (in writing in the notebook) the following - "What sorts of images do CNN classifiers do well with? What sorts do they not do so well? And what are your hypotheses for why?"

### Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)